This notebook is based on notebook [here](https://www.google.com/url?q=https%3A%2F%2Fnbviewer.jupyter.org%2Fgithub%2Fagisga%2Fsciruby-notebooks%2Fblob%2Fmaster%2FData%2520Analysis%2FLogistic%2520regression%2520with%2520categorical%2520data.ipynb&sa=D&sntz=1&usg=AFQjCNE7gDkrVcPcy6d4EeqtRixVhB017A) created by [Alexej](http://github.com/agisga)

# Logistic regression with categorical data

We aim to fit a logistic regression model to the [shelter animal data](https://www.kaggle.com/c/shelter-animal-outcomes) from [kaggle](https://www.kaggle.com/competitions) using the Ruby gems `daru` and `statsample-glm`.



In [1]:
require 'daru'
shelter_data = Daru::DataFrame.from_csv 'data/animal_shelter_train.csv'
shelter_data.head(3)

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"https://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"https://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    w

#<Daru::DataFrame(3x10)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0

In [2]:
shelter_data.to_category 'OutcomeType', 'OutcomeSubtype', 'AnimalType', 'SexuponOutcome', 'Breed', 'Color'
nil

We create a 0-1-valued indicator for whether the animal got adopted.

In [3]:
shelter_data['OutcomeType_Adoption'] = (shelter_data['OutcomeType'].contrast_code)['OutcomeType_Adoption']
shelter_data.head 3

#<Daru::DataFrame(3x11)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut OutcomeTyp
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0          0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0          0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0          1

## Some data preprocessing

### Grouping Breeds

In [4]:
small = shelter_data.head 300
small.head 5

#<Daru::DataFrame(5x11)>
              AnimalID       Name   DateTime OutcomeTyp OutcomeSub AnimalType SexuponOut      Breed      Color AgeuponOut OutcomeTyp
          0    A671945    Hambone 2014-02-12 Return_to_        nil        Dog Neutered M Shetland S Brown/Whit       52.0          0
          1    A656520      Emily 2013-10-13 Euthanasia  Suffering        Cat Spayed Fem Domestic S Cream Tabb       52.0          0
          2    A686464     Pearce 2015-01-31   Adoption     Foster        Dog Neutered M Pit Bull M Blue/White      104.0          1
          3    A683430        nil 2014-07-11   Transfer    Partner        Cat Intact Mal Domestic S Blue Cream        3.0          0
          4    A667013        nil 2013-11-15   Transfer    Partner        Dog Neutered M Lhasa Apso        Tan      104.0          0

In [5]:
small['Breed'].frequencies.sort(ascending: false).head(10)

#<Daru::Vector(10)>
                                     Breed
 Domestic Shorthair M                  104
 Chihuahua Shorthair                    24
         Pit Bull Mix                   18
 Labrador Retriever M                   14
 Domestic Medium Hair                    7
 Domestic Longhair Mi                    6
  German Shepherd Mix                    6
        Dachshund Mix                    5
          Siamese Mix                    4
      Rat Terrier Mix                    3

In [6]:
other_cats = small['Breed'].categories.select { |i| small['Breed'].count(i) < 10 }
other_cats_hash = other_cats.zip(['other']*other_cats.size).to_h
small['Breed'].rename_categories other_cats_hash
small['Breed'].frequencies

#<Daru::Vector(5)>
                                     Breed
 Domestic Shorthair M                  104
         Pit Bull Mix                   18
 Chihuahua Shorthair                    24
 Labrador Retriever M                   14
                other                  140

In [22]:
small['Breed'].base_category = 'other'

"other"

### Grouping colors

In [47]:
p small['Color'].categories.size
small['Color'].frequencies.sort(ascending: false).head 10

366


#<Daru::Vector(10)>
                               Color
       Black/White                35
             Black                27
       Brown Tabby                18
       Brown/White                13
            Calico                12
         Tan/White                11
             White                10
 Brown Tabby/White                 8
       White/Black                 8
      Orange Tabby                 8

In [48]:
other_cats = small['Color'].categories.select { |i| small['Color'].count(i) < 5 }
other_cats_hash = other_cats.zip(['other']*other_cats.size).to_h
small['Color'].rename_categories other_cats_hash
small['Color'].frequencies

#<Daru::Vector(23)>
                                   Color
         Brown/White                  13
                 Tan                   6
           Black/Tan                   5
          Blue Tabby                   5
         Brown Tabby                  18
               White                  10
               Black                  27
               Brown                   8
  Orange Tabby/White                   8
         Black/White                  35
 Brown Brindle/White                   7
         Black/Brown                   6
        Orange Tabby                   8
     Chocolate/White                   6
              Calico                  12
                 ...                 ...

In [49]:
small['Color'].base_category = 'other'

"other"

### Looking at SexuponOutcome

In [68]:
small['SexuponOutcome'].frequencies

#<Daru::Vector(6)>
                SexuponOutcome
  Neutered Male            103
  Spayed Female            109
    Intact Male             38
  Intact Female             39
        Unknown             11
                             0

The last row is weird. Don't know what's causing it. I have to make it go away for regression to work.

In [72]:
p small['SexuponOutcome'].categories
small['SexuponOutcome'].rename_categories nil => 'Unknown'
small['SexuponOutcome'].categories

["Neutered Male", "Spayed Female", "Intact Male", "Intact Female", "Unknown"]


["Neutered Male", "Spayed Female", "Intact Male", "Intact Female", "Unknown"]

## Model fit

Now, having put data in appropriate form, we can fit the logistic regression model with `statsample-glm`.

In [73]:
m = small['OutcomeType_Adoption'].mean
"Trivial accuracy = #{[m, 1-m].max}"

"Trivial accuracy = 0.5900000000000001"

In [77]:
require 'statsample-glm'

formula = 'OutcomeType_Adoption~AnimalType+Breed+AgeuponOutcome(Weeks)+Color+SexuponOutcome'
# formula = 'OutcomeType_Adoption~SexuponOutcome'
# Getting NotRegularMatrix exception if including variables such as Breed and Color.
# See (https://github.com/SciRuby/statsample-glm/issues/32)

glm_adoption = Statsample::GLM::Regression.new formula,
  small, :logistic
glm_adoption.model.coefficients :hash

{:AnimalType_Cat=>0.3624673663127273, :"Breed_Domestic Shorthair Mix"=>-0.6992046526007152, :"Breed_Pit Bull Mix"=>-0.3954385950167811, :"Breed_Chihuahua Shorthair Mix"=>0.05393951646090618, :"Breed_Labrador Retriever Mix"=>0.6709131718767928, :"Color_Brown/White"=>-0.06968015981503961, :Color_Tan=>0.19889260368775816, :"Color_Black/Tan"=>-101.5554199318437, :"Color_Blue Tabby"=>2.7883835881103223, :"Color_Brown Tabby"=>2.401241370703389, :Color_White=>-0.5617805845345925, :Color_Black=>1.7294452972681649, :Color_Brown=>-0.8317841360418532, :"Color_Orange Tabby/White"=>0.9140911618491794, :"Color_Black/White"=>0.6014367036618534, :"Color_Brown Brindle/White"=>-0.24353948921870514, :"Color_Black/Brown"=>-0.06531274323442397, :"Color_Orange Tabby"=>1.924761054499644, :"Color_Chocolate/White"=>0.3324662490477372, :Color_Calico=>-0.2892228353164901, :Color_Tricolor=>-1.2642186282997816, :"Color_White/Black"=>-2.365948424329803, :Color_Tortie=>0.795111541042055, :"Color_Tan/White"=>0.229207

In [78]:
predict = glm_adoption.predict small
predict.map! { |i| i < 0.5 ? 0 : 1 }
predict.head 5

#<Daru::Vector(5)>
   0   1
   1   1
   2   0
   3   0
   4   1

In [79]:
real = small['OutcomeType_Adoption']
correct = real.zip(predict).count { |a, b| a == b }
total = real.size
"Our model accuracy: #{correct/total.to_f}"

"Our model accuracy: 0.7933333333333333"

## Possible next steps

1. Interpret the logistic regression coefficients.
2. Fit logistic regression models with euthanasia, death, etc. as response variable.
3. Predict adoption, euthanasia, death, etc. on test data.
4. Submit prediction results to kaggle, and fail against random forrest models.